<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# Importando as Bibliotecas

In [1]:
# Imports
import os
import boto3
import pandas as pd
import sagemaker
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# Configurações Iniciais no SageMaker

In [2]:
# Define a região
region = boto3.Session().region_name

# Define a role
role = get_execution_role()

# Cria sessão SageMaker
session = sagemaker.Session()

us-east-1


# Carregando os Dados

In [5]:
# Cria o cliente s3
s3 = boto3.client("s3")
bucket_name = 'proj-classification-975050148459'

response = s3.list_objects_v2(Bucket=bucket_name)

if 'Contents' in response:
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print("O bucket está vazio ou não pôde ser acessado.")

In [9]:
# Download do arquivo do bucket para o ambiente local do SageMaker
s3.download_file("proj-classification-975050148459", "dataset.csv", "dataset.csv")

[03/06/25 20:22:40] INFO     Skipping checksum validation. Response did not contain one of the  ]8;id=930023;file:///opt/conda/lib/python3.11/site-packages/botocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=357308;file:///opt/conda/lib/python3.11/site-packages/botocore/httpchecksum.py#481\481]8;;\
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

In [18]:
# Carrega o arquivo como dataframe do pandas
dados = pd.read_csv("./dataset.csv")

# Exploração Inicial dos Dados

In [19]:
# Visualiza as primeiras linhas
dados.head()

,id,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [20]:
# Shape
dados.shape

(344, 9)

In [21]:
# Total de valores ausentes
dados.isna().sum()

id                    0
species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
year                  0
dtype: int64

In [22]:
# Total de duplicatas (complete cases)
dados.duplicated().sum()

0

In [23]:
# Info
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 344 non-null    int64  
 1   species            344 non-null    object 
 2   island             344 non-null    object 
 3   bill_length_mm     342 non-null    float64
 4   bill_depth_mm      342 non-null    float64
 5   flipper_length_mm  342 non-null    float64
 6   body_mass_g        342 non-null    float64
 7   sex                333 non-null    object 
 8   year               344 non-null    int64  
dtypes: float64(4), int64(2), object(3)
memory usage: 24.3+ KB


In [24]:
# Total de valores únicos
dados.nunique()

id                   344
species                3
island                 3
bill_length_mm       164
bill_depth_mm         80
flipper_length_mm     55
body_mass_g           94
sex                    2
year                   3
dtype: int64

In [25]:
# Resumo estatístico
dados.describe()

,id,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,year
count,344.000000,342.000000,342.000000,342.000000,342.000000,344.000000
mean,171.500000,43.921930,17.151170,200.915205,4201.754386,2008.031977
std,99.448479,5.459584,1.974793,14.061714,801.954536,0.816464
min,0.000000,32.100000,13.100000,172.000000,2700.000000,2007.000000
25%,85.750000,39.225000,15.600000,190.000000,3550.000000,2007.000000
50%,171.500000,44.450000,17.300000,197.000000,4050.000000,2008.000000
75%,257.250000,48.500000,18.700000,213.000000,4750.000000,2009.000000
max,343.000000,59.600000,21.500000,231.000000,6300.000000,2009.000000


## Limpeza, Codificação e Preparação dos Dados

In [26]:
# Colunas
dados.columns

Index(['id', 'species', 'island', 'bill_length_mm', 'bill_depth_mm',
       'flipper_length_mm', 'body_mass_g', 'sex', 'year'],
      dtype='object')

In [27]:
print("Categorias na Variável 'species': ", dados['species'].unique())

Categorias na Variável 'species':  ['Adelie' 'Gentoo' 'Chinstrap']


In [28]:
print("Categorias na Variável 'island': ", dados['island'].unique())

Categorias na Variável 'island':  ['Torgersen' 'Biscoe' 'Dream']


In [29]:
# Vamos fazer o drop de variáveis que não serão relevantes para o projeto
dados_limpos = dados.drop(columns = ['id', 'island', 'sex', 'year'])

In [30]:
# Info
dados_limpos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   bill_length_mm     342 non-null    float64
 2   bill_depth_mm      342 non-null    float64
 3   flipper_length_mm  342 non-null    float64
 4   body_mass_g        342 non-null    float64
dtypes: float64(4), object(1)
memory usage: 13.6+ KB


In [31]:
# Vamos fazer o drop de valores ausentes
dados_limpos = dados_limpos.dropna(subset = ['bill_length_mm'])

In [32]:
dados_limpos.isna().sum()

species              0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
dtype: int64

In [33]:
dados_limpos.head()

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
0,Adelie,39.1,18.7,181.0,3750.0
1,Adelie,39.5,17.4,186.0,3800.0
2,Adelie,40.3,18.0,195.0,3250.0
4,Adelie,36.7,19.3,193.0,3450.0
5,Adelie,39.3,20.6,190.0,3650.0


In [34]:
# Label encoding na variável alvo
dados_limpos['species'] = dados_limpos['species'].replace('Adelie', 0)
dados_limpos['species'] = dados_limpos['species'].replace('Gentoo', 1)
dados_limpos['species'] = dados_limpos['species'].replace('Chinstrap', 2)

In [35]:
# Shuffle (embaralhamento dos dados)
dados_limpos = dados_limpos.sample(frac = 1).reset_index(drop = True)

In [36]:
dados_limpos.sample(10)

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
124,0,35.5,16.2,195.0,3350.0
76,0,39.7,18.4,190.0,3900.0
262,0,38.8,17.6,191.0,3275.0
314,1,50.5,15.9,225.0,5400.0
294,0,35.0,17.9,192.0,3725.0
22,0,40.8,18.4,195.0,3900.0
236,0,38.1,17.0,181.0,3175.0
243,0,37.3,17.8,191.0,3350.0
158,1,45.3,13.8,208.0,4200.0
74,1,44.0,13.6,208.0,4350.0


## Divisão dos Dados em Treino e Teste e Armazenamento no S3

In [37]:
# Garante que somente as colunas do shape do dataframe serão usadas 
dados_limpos.columns = range(dados_limpos.shape[1])

In [38]:
dados_limpos.sample(5)

,0,1,2,3,4
40,0,33.1,16.1,178.0,2900.0
221,0,34.0,17.1,185.0,3400.0
305,0,40.2,17.0,176.0,3450.0
34,2,48.1,16.4,199.0,3325.0
192,1,46.5,14.5,213.0,4400.0


In [39]:
# Divisão em treino e validação com proporção 80/20
dados_treino, dados_valid = train_test_split(dados_limpos, test_size = 0.2, random_state = 42)

In [40]:
dados_treino.shape

(273, 5)

In [41]:
dados_valid.shape

(69, 5)

In [42]:
# Nome do bucket S3
bucket_name

'proj-classification-975050148459'

In [43]:
# Salvando e enviando o arquivo de treino
dados_treino.to_csv('dados_treino.csv', header = False, index = False)
key = 'treino/dados_treino'
url = 's3://{}/{}'.format(bucket_name, key)
print(url)

s3://proj-classification-975050148459/treino/dados_treino


In [44]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file('dados_treino.csv')

In [45]:
# Salvando e enviando o arquivo de validação
dados_valid.to_csv('dados_valid.csv', header = False, index = False)
key = 'valid/dados_valid'
url = 's3://{}/{}'.format(bucket_name, key)
print(url)

s3://proj-classification-975050148459/valid/dados_valid


In [46]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file('dados_valid.csv')

## Construção do Modelo

In [47]:
# Nome do bucket S3
bucket_name

'proj-classification-975050148459'

In [48]:
# Nome para salvar o modelo
key = 'model/xgbmodel'

In [49]:
# Local no S3 para salvar o modelo
s3_output_location = f's3://{bucket_name}/{key}'

In [50]:
s3_output_location

's3://proj-classification-975050148459/model/xgbmodel'

In [51]:
# Define a URI da imagem do modelo XGBoost no SageMaker
image_uri = sagemaker.image_uris.retrieve("xgboost", region, version = "1.5-1")

[03/06/25 20:38:09] INFO     Ignoring unnecessary instance type: None.                            ]8;id=267526;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=394235;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#530\530]8;;\

In [62]:
# Cria o estimador XGBoost
xgb_model = sagemaker.estimator.Estimator(image_uri = image_uri,
                                             role = role,
                                             instance_count = 1,
                                             instance_type = 'ml.m4.xlarge',
                                             volume_size = 5,
                                             output_path = s3_output_location,
                                             sagemaker_session = session)

In [63]:
# Configura os hiperparâmetros
xgb_model.set_hyperparameters(max_depth = 5,
                                 gamma = 4,
                                 min_child_weight = 6,
                                 objective = 'multi:softmax',
                                 num_class = 3,
                                 num_round = 8)

## Treinamento do Modelo

In [54]:
bucket_name

'proj-classification-975050148459'

In [55]:
# Define o caminho para os dados
dados_treino = 's3://{}/{}'.format(bucket_name, 'treino/dados_treino')
dados_valid = 's3://{}/{}'.format(bucket_name, 'valid/dados_valid')

In [56]:
dados_treino

's3://proj-classification-975050148459/treino/dados_treino'

In [57]:
dados_valid

's3://proj-classification-975050148459/valid/dados_valid'

In [58]:
# Cria os inputs
input_treino = TrainingInput(s3_data = dados_treino, content_type = 'text/csv')
input_valid = TrainingInput(s3_data = dados_valid, content_type = 'text/csv')

In [60]:
# Define os canais para os dados
canais = {'train': input_treino, 'validation': input_valid}

In [64]:
%%time

xgb_model.fit(inputs = canais)

[03/06/25 20:45:13] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=182485;file:///opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=362145;file:///opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=275425;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=675320;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2025-03-06-20-45-13-466                                             

2025-03-06 20:45:13 Starting - Starting the training job......
..25-03-06 20:46:15 Downloading - Downloading input data.
.....03-06 20:46:45 Downloading - Downloading the training image.
./miniconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[2025-03-06 20:47:57.588 ip-10-2-211-214.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-03-06 20:47:57.613 ip-10-2-211-214.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-03-06:20:47:57:INFO] Imported framework sagemaker_xgboost_container.training
[2025-03-06:20:47:57:INFO] Failed to parse hyperparameter objective value multi:softmax to Json.
Returning the value itself
[2025-03-06:20:47:57:INFO] No GPUs detected (normal if no gpus installed)
[2025-03-06:20:47:58:INFO] Running XGBoost

## Deploy do Modelo com SageMaker

In [ ]:
%%time

xgb_predictor = xgb_model.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

[03/06/25 20:52:43] INFO     Creating model with name: sagemaker-xgboost-2025-03-06-20-52-42-958    ]8;id=889664;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=404890;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4094\4094]8;;\

                    INFO     Creating endpoint-config with name                                     ]8;id=568919;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=31153;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#5889\5889]8;;\
                             sagemaker-xgboost-2025-03-06-20-52-42-958                                             

[03/06/25 20:52:44] INFO     Creating endpoint with name sagemaker-xgboost-2025-03-06-20-52-42-958  ]8;id=959793;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=496450;file:///opt/conda/lib/python3.11/site-packages/sagemaker/session.py#4711\4711]8;;\

---

Ao executar a linha acima, o modelo treinado (dsa_xgbmodel) é implantado em um endpoint gerenciado pelo Amazon SageMaker para inferência em tempo real. Aqui está o que acontece:

**Criação do Endpoint**: O SageMaker cria um endpoint RESTful onde o modelo ficará disponível para receber solicitações de previsão.

**Configuração da Infraestrutura**: É provisionada uma instância do tipo ml.m4.xlarge para hospedar o modelo. Essa instância é dimensionada com base nos requisitos computacionais do modelo.

**Inicialização do Modelo**: O modelo treinado é carregado na instância provisionada, junto com o ambiente necessário para sua execução, como o container do XGBoost.

**Atribuição ao Objeto xgb_predictor**: O objeto xgb_predictor é criado e serve como uma interface para interagir com o endpoint. Ele permite enviar dados para predição e receber as respostas diretamente no ambiente de desenvolvimento.

Este processo disponibiliza o modelo em um ambiente escalável e gerenciado, pronto para realizar inferências em tempo real a partir de requisições enviadas ao endpoint.